In [2]:
# Importing libraries
import json
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import sqlalchemy




# Defining the urls
urls =  ["https://en.wikipedia.org/wiki/Berlin",
        "https://en.wikipedia.org/wiki/Paris",
        "https://en.wikipedia.org/wiki/Madrid"]

# Getting the responses in a list
responses = []

for url in urls:
    responses.append(requests.get(url))


for response in responses:
    if response.status_code != 200:
        print(f"There is an error {response.status_code}")
    else:
        print(f"The response status is: {response.status_code}")
    
# Using BeautifulSoup to get the information from the pages
soups = []

for response in responses:
    soups.append(BS(response.content, 'html.parser'))
    
# Create a data frame
data_df = pd.DataFrame()

# Get the names of the cities
city_names = []
for soup in soups:
    city_names.append(soup.find('span', class_='mw-page-title-main').get_text())
data_df['city'] = city_names

# Get the countries
country_names = []
for soup in soups:
    for s in soup.select("table.infobox tbody tr th"):
        if s.text == "Country":
            country_name = s.next_sibling.text.strip()
            country_names.append(country_name)
            break   

data_df['country'] = country_names

# Get the latitude and longitude
latitude = []
longitude = []
for soup in soups:
    for s in soup.select("table.infobox tbody tr td"):
            if s.text.startswith("Coordinates"):
                lat = s.text.split("/")[-1].split(";")[0]
                lon = s.text.split("/")[-1].split(";")[1]
                latitude.append(lat)
                longitude.append(lon)
                break

data_df['latitude'] = latitude
data_df['longitude'] = longitude

# Get the population
population = []
for soup in soups:
    for s in soup.select("table.infobox tbody tr th"):
        if s.text.startswith("Population"):
            for sibling in s.parent.find_next_siblings():
                if "Metro" in sibling.text:
                    population_text = sibling.select("td")[0].text
                    population.append(population_text.split(" ")[0])
                    break

data_df['population'] = population

# Clean the data
for i, population in enumerate(data_df['population']):
    if ',' in population:
        data_df['population'][i] = population.replace(',', '')


for i, population in enumerate(data_df['population']):
    if '[1]' in population:
        data_df['population'][i] = population.replace('[1]', '')

data_df

# Copy the dataframe to MySQL 
schema="mydb" 
host="127.0.0.1"
user="root"
password="*******"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

data_df.to_sql('cities', 
       if_exists='append',
       con=con,
       index=False)

The response status is: 200
The response status is: 200
The response status is: 200


3